In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units

In [4]:
# load data
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datanew/LW.csv'
df = pd.read_csv(url, delimiter=';')
# input and output
X = df.drop(columns=["GOAL","7MACC","6MACC","9MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
y = df.GOAL
# dataset splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# dataset normalization
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# create the model
model = Sequential()
model.add(Flatten(input_shape=(6,)))  # Adjust the input shape to (12,)
model.add(RBFLayer(6, 0.5))
model.add(Dense(1, activation='linear'))  # Use 'linear' activation for regression
model.compile(optimizer='rmsprop', loss=mean_squared_error)  # Use mean_squared_error for regression
# train model
model.fit(X_train, y_train, batch_size=8, epochs=100)


Epoch 1/100
236/236 [==============================] - 1s 1ms/step - loss: 14.5793
Epoch 2/100
236/236 [==============================] - 0s 1ms/step - loss: 8.2685
Epoch 3/100
236/236 [==============================] - 0s 1ms/step - loss: 6.4571
Epoch 4/100
236/236 [==============================] - 0s 1ms/step - loss: 5.9883
Epoch 5/100
236/236 [==============================] - 0s 1ms/step - loss: 5.6874
Epoch 6/100
236/236 [==============================] - 0s 1ms/step - loss: 5.4149
Epoch 7/100
236/236 [==============================] - 0s 1ms/step - loss: 5.1640
Epoch 8/100
236/236 [==============================] - 0s 1ms/step - loss: 4.9310
Epoch 9/100
236/236 [==============================] - 0s 1ms/step - loss: 4.7069
Epoch 10/100
236/236 [==============================] - 0s 1ms/step - loss: 4.4941
Epoch 11/100
236/236 [==============================] - 0s 1ms/step - loss: 4.2981
Epoch 12/100
236/236 [==============================] - 0s 1ms/step - loss: 4.1058
Epoch 13/100

In [19]:
import numpy as np
new_data = np.array([[5,2,2,3,4,9]])  # Replace value1, value2, ..., value12 with actual values
new_data_normalized = scaler.transform(new_data)
predictions = model.predict(new_data_normalized)
print(predictions)

1/1 [==============================] - 0s 21ms/step
[[2.1381795]]
